In [5]:
import numpy as np
import pandas as pd
import openpyxl 
import os
%run ccp_func.ipynb

In [60]:
def kdpw_colect(quarter):
    #quarter = format: 'Q3-2017'
    
    ccp = 'kdpw'
    
    #basic
    b_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.lower() + '-ccp-' + 
                           quarter.lower() + '.xlsx', sheet_name='AggregatedDataFile')
    #files in use
    a_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.lower() + '-ccp-' +
                           quarter.lower() + '.xlsx', 
                           sheet_name='CCP_DataFile_4_4a')
    c_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.lower() + '-ccp-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name='CCP_DataFile_6_1')
    d_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.lower() + '-ccp-' +
                           quarter.lower() + '.xlsx', 
                           sheet_name='CCP1_DataFile_6.2')
    e_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.lower() + '-ccp-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name='CCP_DataFile_18_2')
    g_file = pd.read_excel('../CCP/data/' + quarter.upper() + '/' + ccp.lower() + '-ccp-' + 
                           quarter.lower() + '.xlsx', 
                           sheet_name='CCP_DataFile_23')
    ex_rate = pd.read_excel('../CCP/data/kursy-walutowe-CCP.xlsx', sheet_name='Arkusz1')
    
    f_file = pd.DataFrame({'CCP':ccp.capitalize()}, index=[0,1])
    f_file['quarter'] = quarter
    f_file['clearning service'] = ['organised market', 'OTC market']
    #all in in in PLN -> need to by recalculate to EUR
    add_ex_rate(b_file, ex_rate, quarter)
    b_file['4.1.1'] = b_file['4.1.1'] // b_file['ex_rate']
    b_file['4.1.2'] = b_file['4.1.2'] // b_file['ex_rate']
    b_file['4.1.3'] = b_file['4.1.3'] // b_file['ex_rate']
    b_file['4.1.4'] = b_file['4.1.4'] // b_file['ex_rate']
    b_file['4.1.5'] = b_file['4.1.5'] // b_file['ex_rate']
    b_file['4.1.6'] = pd.to_numeric(b_file['4.1.6'], errors='coerce') // b_file['ex_rate'] #objects here
    b_file['4.1.7'] = b_file['4.1.7'] // b_file['ex_rate']
    b_file['4.1.8'] = b_file['4.1.8'] // b_file['ex_rate']
    b_file['4.1.9'] = b_file['4.1.9'] // b_file['ex_rate']
    b_file['4.1.10'] = b_file['4.1.10'] // b_file['ex_rate']
    
    add_ex_rate(d_file, ex_rate, quarter)
    a_file['4.4.7'] = a_file['4.4.7'] // d_file['ex_rate']
    
    add_ex_rate(c_file, ex_rate, quarter)
    try:
        c_file['6.1.1'] = c_file['6.1.1'] // c_file['ex_rate']
    except:
        #for Q1-2017
        c_file['6.1.1*'] = c_file['6.1.1*'] // c_file['ex_rate']
    
    add_ex_rate(d_file, ex_rate, quarter)
    d_file['6.2.15'] = d_file['6.2.15'] // d_file['ex_rate']
    
    add_ex_rate(g_file, ex_rate, quarter)
    g_file['23.1.2'] = g_file['23.1.2'] // g_file['ex_rate']
    g_file['23.2.1'] = g_file['23.2.1'] // g_file['ex_rate']
    
    f_file['4.1.1'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.1'].values
    f_file['4.1.2'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.2'].values
    f_file['4.1.3'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.3'].values
    f_file['4.1.4'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.4'].values
    f_file['4.1.5'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.5'].values
    f_file['4.1.6'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.6'].values
    f_file['4.1.7'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.7'].values
    f_file['4.1.8'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.8'].values
    f_file['4.1.9'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.9'].values
    f_file['4.1.10'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['4.1.10'].values
    
    # 4.4.7.PDA & 4.4.7.MA
    f_file['4.4.7.PDA'] = a_file[(a_file.Description == 'PeakDayAmountInPast12Months') & (a_file.ReportLevelIdentifier.isin(['Regulated market', 'OTC market']))].reset_index()['4.4.7']
    f_file['4.4.7.MA'] = a_file[(a_file.Description == 'MeanAverageOverPrevious12Months') & (a_file.ReportLevelIdentifier.isin(['Regulated market', 'OTC market']))].reset_index()['4.4.7']
    
    #6.1.1.CLIENT = 'Client_Net' + 'Client_Gross'
    try:
        ORG_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'Organised market')]['6.1.1'])
        OTC_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'OTC market')]['6.1.1'])
    except:
        #for Q1-2017
        ORG_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'Organised market')]['6.1.1*'])
        OTC_CLIENT= sum(c_file[c_file.Description.isin(['Client_Net', 'Client_Gross']) & (c_file.ReportLevelIdentifier == 'OTC market')]['6.1.1*'])
    
    f_file['6.1.1.CLIENT'] = [ORG_CLIENT, OTC_CLIENT]
    #6.1.1.TOTAL
    try:
        f_file['6.1.1.TOTAL'] = c_file[c_file.Description == 'Total']['6.1.1'].values
    except:
        #for Q1-2017
        f_file['6.1.1.TOTAL'] = c_file[c_file.Description == 'Total']['6.1.1*'].values
    
    #6.2.15.PRE & POST
    f_file['6.2.15.PRE'] = d_file[d_file.Description.isin(['PreHaircut'])]['6.2.15'].values
    f_file['6.2.15.POST'] = d_file[d_file.Description.isin(['PostHaircut'])]['6.2.15'].values

    #number of CCP
    f_file['18.1.1.1'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['18.1.1.1'].values
    f_file['18.1.1.2'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['18.1.1.2'].values
    
    #number od CCP participants (average)
    f_file['18.1.2.2'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['18.1.2.2'].values
    
    #number of clients
    f_file['19.1.1'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['19.1.1'].values
    f_file['19.1.2'] = b_file[b_file.ReportLevelIdentifier.isin(['Clearing Guarantee Fund', 'OTC Guarantee Fund'])]['19.1.2'].values
    
    #23.1.2.OTC & ETD
    ORG_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['Organised market - Cash instruments', 'Organised market - Derivatives'])]['23.1.2'])
    OTC_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['OTC market'])]['23.1.2'])
    f_file['23.1.2.OTC'] = [ORG_OTC, OTC_OTC]
    
    ORG_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['Organised market - Cash instruments', 'Organised market - Derivatives'])]['23.1.2'])
    OTC_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['OTC market'])]['23.1.2'])
    f_file['23.1.2.ETD'] = [ORG_OTC, OTC_OTC]
    
    #23.2.1.OTC & ETD
    ORG_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['Organised market - Cash instruments', 'Organised market - Derivatives'])]['23.2.1'])
    OTC_OTC = sum(g_file[(g_file.Description.isin(['OTC'])) & g_file.ReportLevelIdentifier.isin(['OTC market'])]['23.2.1'])
    f_file['23.2.1.OTC'] = [ORG_OTC, OTC_OTC]
    
    ORG_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['Organised market - Cash instruments', 'Organised market - Derivatives'])]['23.2.1'])
    OTC_OTC = sum(g_file[(g_file.Description.isin(['ETD'])) & g_file.ReportLevelIdentifier.isin(['OTC market'])]['23.2.1'])
    f_file['23.2.1.ETD'] = [ORG_OTC, OTC_OTC]
    
    ccp_save(ccp, quarter, f_file)